
<h1 align=center><font size = 5>Deep Learning and Neural Network with Keras</font></h1>

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
A. <a href="#item31">Build a baseline model</a>  
B. <a href="#item32">Normalize the data</a>  
C. <a href="#item33">Increase the number of epochs</a>  
D. <a href="#item34">Increase the number of hidden layers</a>   
E. <a href="#item35">Report</a>

</font>
</div>

In [28]:
# Import the pandas and the Numpy libraries.
import pandas as pd
import numpy as np

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>

In [29]:
# Download the data and read it into a pandas dataframe.
concrete_data = pd.read_csv('https://cocl.us/concrete_data')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [30]:
# Check numbers of data points
concrete_data.shape

(1030, 9)

In [31]:
# Check missing values
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [32]:
# Check null values
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

#### Split data into predictors and target

The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns except age

In [33]:
predictors = concrete_data.iloc[:,0:7] # all columns except Age and Strength
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5


In [34]:
target = concrete_data['Strength'] # Strength column
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [35]:
n_cols = predictors.shape[1]

<a id='item31'></a>

# A. Build a baseline model

In [36]:
import keras

In [37]:
from keras.models import Sequential
from keras.layers import Dense

In [38]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [39]:
# build the model
model_1h = regression_model()

In [40]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Define a function to build a list of mean squared errors 
# with the model, predictor values and number of epochs as arguments
def calcul_MSE(model, predictor, num_epochs):

    MSE_list = []
    Repetition = 50

    for i in range(Repetition):

        X_train, X_test, y_train, y_test = train_test_split(predictor, target, test_size=0.3, random_state=None)

        # fit the model and train the model on the training data using 50 epochs.
        model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=num_epochs, verbose=2)

        # Make predictions using the testing set
        strength_y_pred = model.predict(X_test)

        # The mean squared error
        MSE = mean_squared_error(y_test, strength_y_pred)
        print(f'--------- MSE {i+1}/{Repetition} = {MSE:.2f}------------------------------------------')
        MSE_list.append(MSE)
    
    return MSE_list

In [41]:
# Build the list of mean squared errors
MSE_list_A = calcul_MSE(model_1h, predictors, 50)

Train on 721 samples, validate on 309 samples
Epoch 1/50
 - 1s - loss: 48951.5840 - val_loss: 35736.4736
Epoch 2/50
 - 0s - loss: 26781.8459 - val_loss: 18307.0959
Epoch 3/50
 - 0s - loss: 13195.9107 - val_loss: 8485.5640
Epoch 4/50
 - 0s - loss: 6033.8145 - val_loss: 3815.9015
Epoch 5/50
 - 0s - loss: 2861.6128 - val_loss: 1952.6459
Epoch 6/50
 - 0s - loss: 1708.5746 - val_loss: 1343.3374
Epoch 7/50
 - 0s - loss: 1351.4389 - val_loss: 1171.5836
Epoch 8/50
 - 0s - loss: 1249.5234 - val_loss: 1110.0980
Epoch 9/50
 - 0s - loss: 1198.6469 - val_loss: 1072.1700
Epoch 10/50
 - 0s - loss: 1160.1071 - val_loss: 1036.1971
Epoch 11/50
 - 0s - loss: 1122.8362 - val_loss: 1001.2197
Epoch 12/50
 - 0s - loss: 1084.7358 - val_loss: 966.7233
Epoch 13/50
 - 0s - loss: 1047.1936 - val_loss: 931.9152
Epoch 14/50
 - 0s - loss: 1010.2526 - val_loss: 896.8683
Epoch 15/50
 - 0s - loss: 972.5266 - val_loss: 862.9312
Epoch 16/50
 - 0s - loss: 936.0878 - val_loss: 830.1491
Epoch 17/50
 - 0s - loss: 900.2197 - 

Epoch 43/50
 - 0s - loss: 163.7837 - val_loss: 167.2302
Epoch 44/50
 - 0s - loss: 163.4149 - val_loss: 167.2068
Epoch 45/50
 - 0s - loss: 163.4387 - val_loss: 167.0944
Epoch 46/50
 - 0s - loss: 163.0399 - val_loss: 166.9396
Epoch 47/50
 - 0s - loss: 162.4431 - val_loss: 167.2912
Epoch 48/50
 - 0s - loss: 163.9567 - val_loss: 166.4973
Epoch 49/50
 - 0s - loss: 163.0175 - val_loss: 167.2316
Epoch 50/50
 - 0s - loss: 166.5332 - val_loss: 166.1579
--------- MSE 3/50 = 166.16------------------------------------------
Train on 721 samples, validate on 309 samples
Epoch 1/50
 - 0s - loss: 160.9624 - val_loss: 168.3030
Epoch 2/50
 - 0s - loss: 161.1983 - val_loss: 168.8764
Epoch 3/50
 - 0s - loss: 160.9935 - val_loss: 168.4898
Epoch 4/50
 - 0s - loss: 162.0299 - val_loss: 168.0331
Epoch 5/50
 - 0s - loss: 160.4544 - val_loss: 169.0395
Epoch 6/50
 - 0s - loss: 159.2409 - val_loss: 167.7007
Epoch 7/50
 - 0s - loss: 160.3230 - val_loss: 168.4497
Epoch 8/50
 - 0s - loss: 159.7993 - val_loss: 168.6

Epoch 34/50
 - 0s - loss: 154.8100 - val_loss: 179.2125
Epoch 35/50
 - 0s - loss: 151.4875 - val_loss: 164.2596
Epoch 36/50
 - 0s - loss: 154.4434 - val_loss: 164.4467
Epoch 37/50
 - 0s - loss: 151.9241 - val_loss: 164.6339
Epoch 38/50
 - 0s - loss: 151.7432 - val_loss: 164.7247
Epoch 39/50
 - 0s - loss: 152.2342 - val_loss: 164.8684
Epoch 40/50
 - 0s - loss: 154.3839 - val_loss: 163.8415
Epoch 41/50
 - 0s - loss: 153.2458 - val_loss: 168.0317
Epoch 42/50
 - 0s - loss: 151.7885 - val_loss: 165.6768
Epoch 43/50
 - 0s - loss: 151.8367 - val_loss: 164.0931
Epoch 44/50
 - 0s - loss: 157.1299 - val_loss: 165.7889
Epoch 45/50
 - 0s - loss: 153.3445 - val_loss: 166.9463
Epoch 46/50
 - 0s - loss: 154.7877 - val_loss: 170.7080
Epoch 47/50
 - 0s - loss: 152.9843 - val_loss: 164.2556
Epoch 48/50
 - 0s - loss: 150.9011 - val_loss: 165.3200
Epoch 49/50
 - 0s - loss: 152.6238 - val_loss: 164.2539
Epoch 50/50
 - 0s - loss: 155.3075 - val_loss: 173.5478
--------- MSE 6/50 = 173.55---------------------

Epoch 25/50
 - 0s - loss: 157.4888 - val_loss: 160.2304
Epoch 26/50
 - 0s - loss: 157.4900 - val_loss: 154.6869
Epoch 27/50
 - 0s - loss: 156.2391 - val_loss: 153.0827
Epoch 28/50
 - 0s - loss: 157.2070 - val_loss: 155.1798
Epoch 29/50
 - 0s - loss: 157.3308 - val_loss: 155.2305
Epoch 30/50
 - 0s - loss: 156.8824 - val_loss: 153.0390
Epoch 31/50
 - 0s - loss: 156.6996 - val_loss: 152.2984
Epoch 32/50
 - 0s - loss: 158.5229 - val_loss: 152.2948
Epoch 33/50
 - 0s - loss: 157.5754 - val_loss: 152.6774
Epoch 34/50
 - 0s - loss: 155.5744 - val_loss: 152.6985
Epoch 35/50
 - 0s - loss: 156.1417 - val_loss: 152.1315
Epoch 36/50
 - 0s - loss: 155.8462 - val_loss: 154.7883
Epoch 37/50
 - 0s - loss: 156.4432 - val_loss: 156.2827
Epoch 38/50
 - 0s - loss: 155.5283 - val_loss: 156.0384
Epoch 39/50
 - 0s - loss: 164.1362 - val_loss: 161.6982
Epoch 40/50
 - 0s - loss: 157.4932 - val_loss: 153.6254
Epoch 41/50
 - 0s - loss: 156.7750 - val_loss: 160.5948
Epoch 42/50
 - 0s - loss: 156.7527 - val_loss: 1

Epoch 16/50
 - 0s - loss: 157.3935 - val_loss: 152.6411
Epoch 17/50
 - 0s - loss: 155.3184 - val_loss: 153.2475
Epoch 18/50
 - 0s - loss: 155.5481 - val_loss: 153.5532
Epoch 19/50
 - 0s - loss: 155.8881 - val_loss: 153.6283
Epoch 20/50
 - 0s - loss: 155.4851 - val_loss: 153.5269
Epoch 21/50
 - 0s - loss: 155.9697 - val_loss: 157.9330
Epoch 22/50
 - 0s - loss: 154.7932 - val_loss: 153.3217
Epoch 23/50
 - 0s - loss: 154.5823 - val_loss: 160.2960
Epoch 24/50
 - 0s - loss: 158.9073 - val_loss: 153.6106
Epoch 25/50
 - 0s - loss: 154.9136 - val_loss: 161.5198
Epoch 26/50
 - 0s - loss: 155.7134 - val_loss: 155.0240
Epoch 27/50
 - 0s - loss: 156.6833 - val_loss: 156.1093
Epoch 28/50
 - 0s - loss: 155.7091 - val_loss: 161.0109
Epoch 29/50
 - 0s - loss: 159.2847 - val_loss: 153.6632
Epoch 30/50
 - 0s - loss: 157.1903 - val_loss: 153.0824
Epoch 31/50
 - 0s - loss: 155.4716 - val_loss: 154.3031
Epoch 32/50
 - 0s - loss: 154.4420 - val_loss: 155.6240
Epoch 33/50
 - 0s - loss: 156.5818 - val_loss: 1

...

Epoch 39/50
 - 0s - loss: 157.9799 - val_loss: 147.4735
Epoch 40/50
 - 0s - loss: 158.7565 - val_loss: 148.6614
Epoch 41/50
 - 0s - loss: 159.8853 - val_loss: 147.2655
Epoch 42/50
 - 0s - loss: 160.0633 - val_loss: 147.5455
Epoch 43/50
 - 0s - loss: 158.8295 - val_loss: 147.6128
Epoch 44/50
 - 0s - loss: 160.5165 - val_loss: 151.1780
Epoch 45/50
 - 0s - loss: 160.4398 - val_loss: 148.0667
Epoch 46/50
 - 0s - loss: 158.7440 - val_loss: 147.2394
Epoch 47/50
 - 0s - loss: 158.5291 - val_loss: 147.1618
Epoch 48/50
 - 0s - loss: 158.0094 - val_loss: 146.9480
Epoch 49/50
 - 0s - loss: 158.2555 - val_loss: 147.9655
Epoch 50/50
 - 0s - loss: 158.4233 - val_loss: 147.9238
--------- MSE 37/50 = 147.92------------------------------------------
Train on 721 samples, validate on 309 samples
Epoch 1/50
 - 0s - loss: 157.8355 - val_loss: 153.1403
Epoch 2/50
 - 0s - loss: 159.5011 - val_loss: 149.9744
Epoch 3/50
 - 0s - loss: 160.3545 - val_loss: 150.0206
Epoch 4/50
 - 0s - loss: 155.7868 - val_loss: 

Epoch 30/50
 - 0s - loss: 157.7324 - val_loss: 147.8302
Epoch 31/50
 - 0s - loss: 158.1662 - val_loss: 147.7572
Epoch 32/50
 - 0s - loss: 159.1779 - val_loss: 152.8546
Epoch 33/50
 - 0s - loss: 159.4967 - val_loss: 154.7415
Epoch 34/50
 - 0s - loss: 160.5295 - val_loss: 148.4508
Epoch 35/50
 - 0s - loss: 158.2400 - val_loss: 147.9890
Epoch 36/50
 - 0s - loss: 158.3930 - val_loss: 149.8171
Epoch 37/50
 - 0s - loss: 158.0280 - val_loss: 147.2028
Epoch 38/50
 - 0s - loss: 158.2007 - val_loss: 147.3699
Epoch 39/50
 - 0s - loss: 158.4119 - val_loss: 147.3700
Epoch 40/50
 - 0s - loss: 159.2861 - val_loss: 150.2322
Epoch 41/50
 - 0s - loss: 160.9195 - val_loss: 155.8600
Epoch 42/50
 - 0s - loss: 160.4478 - val_loss: 148.1899
Epoch 43/50
 - 0s - loss: 158.2194 - val_loss: 147.3373
Epoch 44/50
 - 0s - loss: 157.8512 - val_loss: 150.5891
Epoch 45/50
 - 0s - loss: 159.2910 - val_loss: 154.1460
Epoch 46/50
 - 0s - loss: 158.7241 - val_loss: 147.5043
Epoch 47/50
 - 0s - loss: 157.9258 - val_loss: 1

Epoch 21/50
 - 0s - loss: 155.1119 - val_loss: 155.7402
Epoch 22/50
 - 0s - loss: 154.8674 - val_loss: 154.3482
Epoch 23/50
 - 0s - loss: 155.2951 - val_loss: 156.9963
Epoch 24/50
 - 0s - loss: 155.5284 - val_loss: 154.6055
Epoch 25/50
 - 0s - loss: 155.8917 - val_loss: 153.9327
Epoch 26/50
 - 0s - loss: 154.7919 - val_loss: 155.0909
Epoch 27/50
 - 0s - loss: 154.2250 - val_loss: 153.2119
Epoch 28/50
 - 0s - loss: 159.0114 - val_loss: 153.9804
Epoch 29/50
 - 0s - loss: 156.8899 - val_loss: 154.2639
Epoch 30/50
 - 0s - loss: 154.7641 - val_loss: 155.2778
Epoch 31/50
 - 0s - loss: 154.9642 - val_loss: 161.0753
Epoch 32/50
 - 0s - loss: 154.7822 - val_loss: 154.3165
Epoch 33/50
 - 0s - loss: 155.2419 - val_loss: 153.4498
Epoch 34/50
 - 0s - loss: 154.6449 - val_loss: 160.9223
Epoch 35/50
 - 0s - loss: 155.1036 - val_loss: 158.1233
Epoch 36/50
 - 0s - loss: 155.5814 - val_loss: 153.8895
Epoch 37/50
 - 0s - loss: 157.4336 - val_loss: 156.6245
Epoch 38/50
 - 0s - loss: 155.8426 - val_loss: 1

Epoch 12/50
 - 0s - loss: 154.4739 - val_loss: 167.7893
Epoch 13/50
 - 0s - loss: 155.6061 - val_loss: 155.9797
Epoch 14/50
 - 0s - loss: 155.8478 - val_loss: 161.1694
Epoch 15/50
 - 0s - loss: 155.4154 - val_loss: 157.9931
Epoch 16/50
 - 0s - loss: 155.5193 - val_loss: 163.4706
Epoch 17/50
 - 0s - loss: 157.1709 - val_loss: 157.1550
Epoch 18/50
 - 0s - loss: 156.0462 - val_loss: 156.3105
Epoch 19/50
 - 0s - loss: 154.0260 - val_loss: 157.6454
Epoch 20/50
 - 0s - loss: 153.4476 - val_loss: 162.5522
Epoch 21/50
 - 0s - loss: 155.8273 - val_loss: 167.3189
Epoch 22/50
 - 0s - loss: 157.1728 - val_loss: 163.2729
Epoch 23/50
 - 0s - loss: 155.1701 - val_loss: 158.8384
Epoch 24/50
 - 0s - loss: 158.9762 - val_loss: 158.7569
Epoch 25/50
 - 0s - loss: 155.5518 - val_loss: 157.4004
Epoch 26/50
 - 0s - loss: 153.5302 - val_loss: 157.8687
Epoch 27/50
 - 0s - loss: 151.6459 - val_loss: 175.0724
Epoch 28/50
 - 0s - loss: 157.2518 - val_loss: 169.0708
Epoch 29/50
 - 0s - loss: 155.4277 - val_loss: 1

Epoch 3/50
 - 0s - loss: 159.1409 - val_loss: 149.4662
Epoch 4/50
 - 0s - loss: 159.8073 - val_loss: 153.8755
Epoch 5/50
 - 0s - loss: 161.9151 - val_loss: 150.6647
Epoch 6/50
 - 0s - loss: 158.7152 - val_loss: 152.1881
Epoch 7/50
 - 0s - loss: 157.5918 - val_loss: 150.0596
Epoch 8/50
 - 0s - loss: 157.9051 - val_loss: 150.7027
Epoch 9/50
 - 0s - loss: 157.8661 - val_loss: 152.7743
Epoch 10/50
 - 0s - loss: 158.5771 - val_loss: 153.8663
Epoch 11/50
 - 0s - loss: 159.0851 - val_loss: 150.3087
Epoch 12/50
 - 0s - loss: 157.9736 - val_loss: 151.6233
Epoch 13/50
 - 0s - loss: 160.7921 - val_loss: 151.4867
Epoch 14/50
 - 0s - loss: 160.8651 - val_loss: 151.9268
Epoch 15/50
 - 0s - loss: 158.3332 - val_loss: 151.1261
Epoch 16/50
 - 0s - loss: 158.1081 - val_loss: 151.9030
Epoch 17/50
 - 0s - loss: 157.2359 - val_loss: 153.1363
Epoch 18/50
 - 0s - loss: 156.9967 - val_loss: 152.0340
Epoch 19/50
 - 0s - loss: 161.5227 - val_loss: 151.3596
Epoch 20/50
 - 0s - loss: 158.6080 - val_loss: 151.7984

In [42]:
# Calculate the length, the mean and the standard deviation
length_MSE_A = len(MSE_list_A)
mean_MSE_A = np.mean(MSE_list_A)
sd_MSE_A = np.std(MSE_list_A)

In [43]:
# Print the values
print(f'Number of mean squared errors: {length_MSE_A}')
print(f'Mean of the mean squared errors: {mean_MSE_A:.2f}')
print(f'Standard deviation of the mean squared errors: {sd_MSE_A:.2f}')

Number of mean squared errors: 50
Mean of the mean squared errors: 157.51
Standard deviation of the mean squared errors: 16.01


<a id='item32'></a>

## B. Normalize the data

Repeat Part A but use a normalized version of the data. Recall that one way to normalize the data is by subtracting the mean from the individual predictors and dividing by the standard deviation.

In [44]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569


In [45]:
# Build the list of mean squared errors
MSE_list_B = calcul_MSE(model_1h, predictors_norm, 50)

Train on 721 samples, validate on 309 samples
Epoch 1/50
 - 0s - loss: 1402.3081 - val_loss: 1508.2592
Epoch 2/50
 - 0s - loss: 1298.7292 - val_loss: 1394.7003
Epoch 3/50
 - 0s - loss: 1190.6479 - val_loss: 1277.5587
Epoch 4/50
 - 0s - loss: 1083.6923 - val_loss: 1162.0797
Epoch 5/50
 - 0s - loss: 981.4490 - val_loss: 1054.6450
Epoch 6/50
 - 0s - loss: 886.2062 - val_loss: 957.7850
Epoch 7/50
 - 0s - loss: 800.6218 - val_loss: 867.9921
Epoch 8/50
 - 0s - loss: 724.4761 - val_loss: 788.4345
Epoch 9/50
 - 0s - loss: 657.7065 - val_loss: 719.4013
Epoch 10/50
 - 0s - loss: 600.1883 - val_loss: 658.0441
Epoch 11/50
 - 0s - loss: 549.1955 - val_loss: 606.0664
Epoch 12/50
 - 0s - loss: 506.5038 - val_loss: 558.7610
Epoch 13/50
 - 0s - loss: 468.2936 - val_loss: 519.4729
Epoch 14/50
 - 0s - loss: 436.2737 - val_loss: 484.3840
Epoch 15/50
 - 0s - loss: 408.5690 - val_loss: 453.4680
Epoch 16/50
 - 0s - loss: 384.2426 - val_loss: 426.9396
Epoch 17/50
 - 0s - loss: 363.1590 - val_loss: 402.7659
Ep

Epoch 43/50
 - 0s - loss: 157.3024 - val_loss: 157.4171
Epoch 44/50
 - 0s - loss: 157.0685 - val_loss: 157.3693
Epoch 45/50
 - 0s - loss: 156.9238 - val_loss: 157.4753
Epoch 46/50
 - 0s - loss: 156.8786 - val_loss: 157.5306
Epoch 47/50
 - 0s - loss: 156.7653 - val_loss: 157.2846
Epoch 48/50
 - 0s - loss: 156.6146 - val_loss: 157.2720
Epoch 49/50
 - 0s - loss: 156.5215 - val_loss: 157.0416
Epoch 50/50
 - 0s - loss: 156.4212 - val_loss: 157.0001
--------- MSE 3/50 = 157.00------------------------------------------
Train on 721 samples, validate on 309 samples
Epoch 1/50
 - 0s - loss: 157.0805 - val_loss: 155.0139
Epoch 2/50
 - 0s - loss: 156.6567 - val_loss: 155.0819
Epoch 3/50
 - 0s - loss: 156.2875 - val_loss: 155.2037
Epoch 4/50
 - 0s - loss: 155.9590 - val_loss: 155.3232
Epoch 5/50
 - 0s - loss: 155.8124 - val_loss: 155.4249
Epoch 6/50
 - 0s - loss: 155.6351 - val_loss: 155.3212
Epoch 7/50
 - 0s - loss: 155.4948 - val_loss: 155.3978
Epoch 8/50
 - 0s - loss: 155.3414 - val_loss: 155.4

Epoch 34/50
 - 0s - loss: 145.4200 - val_loss: 157.1436
Epoch 35/50
 - 0s - loss: 145.3958 - val_loss: 157.2623
Epoch 36/50
 - 0s - loss: 145.3406 - val_loss: 157.0663
Epoch 37/50
 - 0s - loss: 145.2791 - val_loss: 157.2963
Epoch 38/50
 - 0s - loss: 145.2908 - val_loss: 157.4447
Epoch 39/50
 - 0s - loss: 145.2060 - val_loss: 157.1436
Epoch 40/50
 - 0s - loss: 145.2320 - val_loss: 157.0118
Epoch 41/50
 - 0s - loss: 145.2164 - val_loss: 156.9795
Epoch 42/50
 - 0s - loss: 145.1943 - val_loss: 156.9885
Epoch 43/50
 - 0s - loss: 145.0973 - val_loss: 157.1526
Epoch 44/50
 - 0s - loss: 145.0464 - val_loss: 157.0433
Epoch 45/50
 - 0s - loss: 145.0648 - val_loss: 157.1228
Epoch 46/50
 - 0s - loss: 145.0090 - val_loss: 157.1824
Epoch 47/50
 - 0s - loss: 144.9586 - val_loss: 157.0435
Epoch 48/50
 - 0s - loss: 145.0159 - val_loss: 157.0685
Epoch 49/50
 - 0s - loss: 144.9105 - val_loss: 156.9630
Epoch 50/50
 - 0s - loss: 144.8802 - val_loss: 156.7969
--------- MSE 6/50 = 156.80---------------------

Epoch 25/50
 - 0s - loss: 146.6274 - val_loss: 138.0445
Epoch 26/50
 - 0s - loss: 146.5683 - val_loss: 138.0126
Epoch 27/50
 - 0s - loss: 146.5242 - val_loss: 138.0222
Epoch 28/50
 - 0s - loss: 146.4694 - val_loss: 138.1645
Epoch 29/50
 - 0s - loss: 146.4352 - val_loss: 138.1663
Epoch 30/50
 - 0s - loss: 146.4270 - val_loss: 138.1345
Epoch 31/50
 - 0s - loss: 146.3548 - val_loss: 137.8279
Epoch 32/50
 - 0s - loss: 146.2022 - val_loss: 137.9277
Epoch 33/50
 - 0s - loss: 146.2075 - val_loss: 137.7337
Epoch 34/50
 - 0s - loss: 146.1073 - val_loss: 137.9189
Epoch 35/50
 - 0s - loss: 146.0584 - val_loss: 138.2570
Epoch 36/50
 - 0s - loss: 146.0681 - val_loss: 137.9404
Epoch 37/50
 - 0s - loss: 145.9555 - val_loss: 137.9281
Epoch 38/50
 - 0s - loss: 145.8849 - val_loss: 137.7633
Epoch 39/50
 - 0s - loss: 145.8234 - val_loss: 138.0917
Epoch 40/50
 - 0s - loss: 145.7676 - val_loss: 138.0446
Epoch 41/50
 - 0s - loss: 145.7226 - val_loss: 137.8643
Epoch 42/50
 - 0s - loss: 145.5630 - val_loss: 1

Epoch 16/50
 - 0s - loss: 132.9147 - val_loss: 150.5790
Epoch 17/50
 - 0s - loss: 132.8153 - val_loss: 150.7336
Epoch 18/50
 - 0s - loss: 132.8003 - val_loss: 150.3958
Epoch 19/50
 - 0s - loss: 132.8694 - val_loss: 150.9664
Epoch 20/50
 - 0s - loss: 132.7627 - val_loss: 150.5999
Epoch 21/50
 - 0s - loss: 132.6948 - val_loss: 150.8044
Epoch 22/50
 - 0s - loss: 132.7417 - val_loss: 150.6809
Epoch 23/50
 - 0s - loss: 132.6518 - val_loss: 150.7244
Epoch 24/50
 - 0s - loss: 132.5833 - val_loss: 150.6786
Epoch 25/50
 - 0s - loss: 132.5777 - val_loss: 150.9035
Epoch 26/50
 - 0s - loss: 132.5372 - val_loss: 150.8872
Epoch 27/50
 - 0s - loss: 132.4962 - val_loss: 151.0808
Epoch 28/50
 - 0s - loss: 132.4555 - val_loss: 150.9395
Epoch 29/50
 - 0s - loss: 132.5511 - val_loss: 150.6474
Epoch 30/50
 - 0s - loss: 132.3530 - val_loss: 150.7620
Epoch 31/50
 - 0s - loss: 132.3631 - val_loss: 150.9100
Epoch 32/50
 - 0s - loss: 132.3099 - val_loss: 150.9121
Epoch 33/50
 - 0s - loss: 132.3812 - val_loss: 1

...

Epoch 39/50
 - 0s - loss: 123.9519 - val_loss: 144.6497
Epoch 40/50
 - 0s - loss: 123.9211 - val_loss: 144.4560
Epoch 41/50
 - 0s - loss: 123.9140 - val_loss: 144.8475
Epoch 42/50
 - 0s - loss: 123.9010 - val_loss: 144.7522
Epoch 43/50
 - 0s - loss: 123.8470 - val_loss: 144.7687
Epoch 44/50
 - 0s - loss: 123.8953 - val_loss: 144.9185
Epoch 45/50
 - 0s - loss: 123.8759 - val_loss: 144.6419
Epoch 46/50
 - 0s - loss: 123.8435 - val_loss: 144.6919
Epoch 47/50
 - 0s - loss: 123.8967 - val_loss: 144.8010
Epoch 48/50
 - 0s - loss: 123.9252 - val_loss: 144.8248
Epoch 49/50
 - 0s - loss: 123.8146 - val_loss: 144.9382
Epoch 50/50
 - 0s - loss: 123.8572 - val_loss: 144.9958
--------- MSE 37/50 = 145.00------------------------------------------
Train on 721 samples, validate on 309 samples
Epoch 1/50
 - 0s - loss: 129.6395 - val_loss: 131.2130
Epoch 2/50
 - 0s - loss: 129.4398 - val_loss: 131.1966
Epoch 3/50
 - 0s - loss: 129.1958 - val_loss: 131.2785
Epoch 4/50
 - 0s - loss: 129.1627 - val_loss: 

Epoch 30/50
 - 0s - loss: 127.9811 - val_loss: 133.2028
Epoch 31/50
 - 0s - loss: 128.0472 - val_loss: 133.3473
Epoch 32/50
 - 0s - loss: 127.9374 - val_loss: 133.4891
Epoch 33/50
 - 0s - loss: 127.8810 - val_loss: 133.5786
Epoch 34/50
 - 0s - loss: 127.9142 - val_loss: 133.6548
Epoch 35/50
 - 0s - loss: 127.8752 - val_loss: 133.6472
Epoch 36/50
 - 0s - loss: 128.0052 - val_loss: 133.4714
Epoch 37/50
 - 0s - loss: 127.9741 - val_loss: 133.5736
Epoch 38/50
 - 0s - loss: 127.8701 - val_loss: 133.4601
Epoch 39/50
 - 0s - loss: 127.8412 - val_loss: 133.4544
Epoch 40/50
 - 0s - loss: 127.8378 - val_loss: 133.5474
Epoch 41/50
 - 0s - loss: 127.8676 - val_loss: 133.8045
Epoch 42/50
 - 0s - loss: 127.8536 - val_loss: 133.6676
Epoch 43/50
 - 0s - loss: 127.8757 - val_loss: 133.5658
Epoch 44/50
 - 0s - loss: 127.7735 - val_loss: 133.8084
Epoch 45/50
 - 0s - loss: 127.8189 - val_loss: 133.8697
Epoch 46/50
 - 0s - loss: 127.7763 - val_loss: 133.9070
Epoch 47/50
 - 0s - loss: 127.7605 - val_loss: 1

Epoch 21/50
 - 0s - loss: 118.3722 - val_loss: 154.7995
Epoch 22/50
 - 0s - loss: 118.3701 - val_loss: 154.7034
Epoch 23/50
 - 0s - loss: 118.3966 - val_loss: 154.7556
Epoch 24/50
 - 0s - loss: 118.4584 - val_loss: 154.8700
Epoch 25/50
 - 0s - loss: 118.3590 - val_loss: 154.9335
Epoch 26/50
 - 0s - loss: 118.3961 - val_loss: 154.9398
Epoch 27/50
 - 0s - loss: 118.4012 - val_loss: 154.8671
Epoch 28/50
 - 0s - loss: 118.3389 - val_loss: 154.9305
Epoch 29/50
 - 0s - loss: 118.3672 - val_loss: 155.0046
Epoch 30/50
 - 0s - loss: 118.4610 - val_loss: 155.0151
Epoch 31/50
 - 0s - loss: 118.3084 - val_loss: 154.9857
Epoch 32/50
 - 0s - loss: 118.2638 - val_loss: 154.8952
Epoch 33/50
 - 0s - loss: 118.2661 - val_loss: 155.1314
Epoch 34/50
 - 0s - loss: 118.2834 - val_loss: 155.0582
Epoch 35/50
 - 0s - loss: 118.2843 - val_loss: 154.9747
Epoch 36/50
 - 0s - loss: 118.2862 - val_loss: 155.1286
Epoch 37/50
 - 0s - loss: 118.2029 - val_loss: 155.1296
Epoch 38/50
 - 0s - loss: 118.2989 - val_loss: 1

Epoch 12/50
 - 0s - loss: 123.4863 - val_loss: 143.0615
Epoch 13/50
 - 0s - loss: 123.5275 - val_loss: 142.9188
Epoch 14/50
 - 0s - loss: 123.4931 - val_loss: 143.1732
Epoch 15/50
 - 0s - loss: 123.5007 - val_loss: 143.3202
Epoch 16/50
 - 0s - loss: 123.4471 - val_loss: 143.3793
Epoch 17/50
 - 0s - loss: 123.4589 - val_loss: 143.4406
Epoch 18/50
 - 0s - loss: 123.3828 - val_loss: 143.5152
Epoch 19/50
 - 0s - loss: 123.4625 - val_loss: 143.6893
Epoch 20/50
 - 0s - loss: 123.3214 - val_loss: 143.7668
Epoch 21/50
 - 0s - loss: 123.3978 - val_loss: 143.7036
Epoch 22/50
 - 0s - loss: 123.4053 - val_loss: 143.6734
Epoch 23/50
 - 0s - loss: 123.3529 - val_loss: 143.8066
Epoch 24/50
 - 0s - loss: 123.2782 - val_loss: 143.7287
Epoch 25/50
 - 0s - loss: 123.2856 - val_loss: 143.9089
Epoch 26/50
 - 0s - loss: 123.2028 - val_loss: 143.9145
Epoch 27/50
 - 0s - loss: 123.2661 - val_loss: 144.0276
Epoch 28/50
 - 0s - loss: 123.2557 - val_loss: 144.1635
Epoch 29/50
 - 0s - loss: 123.2651 - val_loss: 1

Epoch 3/50
 - 0s - loss: 122.7864 - val_loss: 143.9891
Epoch 4/50
 - 0s - loss: 122.7539 - val_loss: 144.3828
Epoch 5/50
 - 0s - loss: 122.7596 - val_loss: 144.2942
Epoch 6/50
 - 0s - loss: 122.7228 - val_loss: 144.6407
Epoch 7/50
 - 0s - loss: 122.7071 - val_loss: 144.5428
Epoch 8/50
 - 0s - loss: 122.6520 - val_loss: 144.3713
Epoch 9/50
 - 0s - loss: 122.5749 - val_loss: 144.5268
Epoch 10/50
 - 0s - loss: 122.6069 - val_loss: 144.6171
Epoch 11/50
 - 0s - loss: 122.6981 - val_loss: 144.6442
Epoch 12/50
 - 0s - loss: 122.5758 - val_loss: 144.6498
Epoch 13/50
 - 0s - loss: 122.7441 - val_loss: 144.8937
Epoch 14/50
 - 0s - loss: 122.8075 - val_loss: 144.4791
Epoch 15/50
 - 0s - loss: 122.5295 - val_loss: 144.8147
Epoch 16/50
 - 0s - loss: 122.5585 - val_loss: 144.9856
Epoch 17/50
 - 0s - loss: 122.5674 - val_loss: 144.8444
Epoch 18/50
 - 0s - loss: 122.5727 - val_loss: 144.7892
Epoch 19/50
 - 0s - loss: 122.5924 - val_loss: 145.1491
Epoch 20/50
 - 0s - loss: 122.5429 - val_loss: 144.9271

In [46]:
# Calculate the length, the mean and the standard deviation
length_MSE_B = len(MSE_list_B)
mean_MSE_B = np.mean(MSE_list_B)
sd_MSE_B = np.std(MSE_list_B)

In [47]:
# Print the values
print(f'Number of mean squared errors: {length_MSE_B}')
print(f'Mean of the mean squared errors: {mean_MSE_B:.2f}')
print(f'Standard deviation of the mean squared errors: {sd_MSE_B:.2f}')

Number of mean squared errors: 50
Mean of the mean squared errors: 139.95
Standard deviation of the mean squared errors: 16.01


In [48]:
# DataFrame
data = {'A':[mean_MSE_A , sd_MSE_A], 'B':[mean_MSE_B , sd_MSE_B]} 
  
# Creates pandas DataFrame. 
df = pd.DataFrame(data, index =['Mean', 'StD'])
  
# print the data 
df.round(2)

,A,B
Mean,157.51,139.95
StD,16.01,16.01


__How does the mean of the mean squared errors compare to that from Step A?__ 

The mean from step B is lower so the model is better. Normalizing the date allow the predictions to be more accurate. 

<a id='item33'></a>

## C. Increase the number of epochs

Repeat Part B but use 100 epochs this time for training.

In [49]:
# Build the list of mean squared errors
MSE_list_C = calcul_MSE(model_1h, predictors_norm, 100)

Train on 721 samples, validate on 309 samples
Epoch 1/100
 - 0s - loss: 129.6738 - val_loss: 128.6527
Epoch 2/100
 - 0s - loss: 129.3740 - val_loss: 128.8975
Epoch 3/100
 - 0s - loss: 129.2954 - val_loss: 129.5317
Epoch 4/100
 - 0s - loss: 129.1198 - val_loss: 129.6501
Epoch 5/100
 - 0s - loss: 129.0625 - val_loss: 129.5815
Epoch 6/100
 - 0s - loss: 129.0803 - val_loss: 130.0876
Epoch 7/100
 - 0s - loss: 128.9792 - val_loss: 129.7399
Epoch 8/100
 - 0s - loss: 128.9873 - val_loss: 129.9992
Epoch 9/100
 - 0s - loss: 128.8975 - val_loss: 130.0138
Epoch 10/100
 - 0s - loss: 128.9124 - val_loss: 129.9020
Epoch 11/100
 - 0s - loss: 128.8376 - val_loss: 130.3066
Epoch 12/100
 - 0s - loss: 128.8534 - val_loss: 130.2769
Epoch 13/100
 - 0s - loss: 128.8256 - val_loss: 130.2647
Epoch 14/100
 - 0s - loss: 128.8219 - val_loss: 130.1957
Epoch 15/100
 - 0s - loss: 128.9116 - val_loss: 130.1902
Epoch 16/100
 - 0s - loss: 128.7621 - val_loss: 130.1562
Epoch 17/100
 - 0s - loss: 128.8590 - val_loss: 130

 - 0s - loss: 126.8210 - val_loss: 135.1519
Epoch 43/100
 - 0s - loss: 126.6779 - val_loss: 134.8230
Epoch 44/100
 - 0s - loss: 126.7317 - val_loss: 134.8886
Epoch 45/100
 - 0s - loss: 126.7217 - val_loss: 134.8674
Epoch 46/100
 - 0s - loss: 126.7755 - val_loss: 135.1004
Epoch 47/100
 - 0s - loss: 126.7468 - val_loss: 134.8879
Epoch 48/100
 - 0s - loss: 126.7382 - val_loss: 134.9210
Epoch 49/100
 - 0s - loss: 126.6381 - val_loss: 134.8301
Epoch 50/100
 - 0s - loss: 126.6074 - val_loss: 135.0190
Epoch 51/100
 - 0s - loss: 126.7377 - val_loss: 135.0101
Epoch 52/100
 - 0s - loss: 126.6861 - val_loss: 134.9022
Epoch 53/100
 - 0s - loss: 126.7106 - val_loss: 134.9993
Epoch 54/100
 - 0s - loss: 126.6316 - val_loss: 135.0581
Epoch 55/100
 - 0s - loss: 126.6360 - val_loss: 135.2269
Epoch 56/100
 - 0s - loss: 126.8285 - val_loss: 135.0606
Epoch 57/100
 - 0s - loss: 126.5963 - val_loss: 135.1457
Epoch 58/100
 - 0s - loss: 126.5780 - val_loss: 135.2274
Epoch 59/100
 - 0s - loss: 126.6266 - val_lo

 - 0s - loss: 128.4638 - val_loss: 130.3633
Epoch 85/100
 - 0s - loss: 128.4500 - val_loss: 130.2339
Epoch 86/100
 - 0s - loss: 128.3508 - val_loss: 130.4115
Epoch 87/100
 - 0s - loss: 128.3963 - val_loss: 130.3818
Epoch 88/100
 - 0s - loss: 128.4275 - val_loss: 130.4090
Epoch 89/100
 - 0s - loss: 128.3336 - val_loss: 130.3481
Epoch 90/100
 - 0s - loss: 128.4135 - val_loss: 130.3296
Epoch 91/100
 - 0s - loss: 128.3697 - val_loss: 130.3856
Epoch 92/100
 - 0s - loss: 128.4284 - val_loss: 130.4704
Epoch 93/100
 - 0s - loss: 128.3739 - val_loss: 130.4700
Epoch 94/100
 - 0s - loss: 128.3385 - val_loss: 130.4797
Epoch 95/100
 - 0s - loss: 128.3479 - val_loss: 130.4631
Epoch 96/100
 - 0s - loss: 128.3948 - val_loss: 130.7202
Epoch 97/100
 - 0s - loss: 128.3375 - val_loss: 130.4420
Epoch 98/100
 - 0s - loss: 128.4149 - val_loss: 130.7640
Epoch 99/100
 - 0s - loss: 128.4683 - val_loss: 130.5951
Epoch 100/100
 - 0s - loss: 128.3016 - val_loss: 130.4865
--------- MSE 3/50 = 130.49----------------

 - 0s - loss: 129.3287 - val_loss: 127.5882
Epoch 25/100
 - 0s - loss: 129.4371 - val_loss: 127.8245
Epoch 26/100
 - 0s - loss: 129.3576 - val_loss: 127.6564
Epoch 27/100
 - 0s - loss: 129.3057 - val_loss: 127.8676
Epoch 28/100
 - 0s - loss: 129.2828 - val_loss: 127.7652
Epoch 29/100
 - 0s - loss: 129.3994 - val_loss: 128.0181
Epoch 30/100
 - 0s - loss: 129.3524 - val_loss: 127.8704
Epoch 31/100
 - 0s - loss: 129.3017 - val_loss: 127.9003
Epoch 32/100
 - 0s - loss: 129.2452 - val_loss: 127.9359
Epoch 33/100
 - 0s - loss: 129.3888 - val_loss: 127.9663
Epoch 34/100
 - 0s - loss: 129.2647 - val_loss: 128.0188
Epoch 35/100
 - 0s - loss: 129.2786 - val_loss: 128.0931
Epoch 36/100
 - 0s - loss: 129.1913 - val_loss: 128.1482
Epoch 37/100
 - 0s - loss: 129.1850 - val_loss: 128.0575
Epoch 38/100
 - 0s - loss: 129.1959 - val_loss: 128.0776
Epoch 39/100
 - 0s - loss: 129.3204 - val_loss: 128.0283
Epoch 40/100
 - 0s - loss: 129.1696 - val_loss: 128.0780
Epoch 41/100
 - 0s - loss: 129.1809 - val_lo

 - 0s - loss: 130.0269 - val_loss: 126.3554
Epoch 67/100
 - 0s - loss: 129.9899 - val_loss: 126.3757
Epoch 68/100
 - 0s - loss: 129.9692 - val_loss: 126.5069
Epoch 69/100
 - 0s - loss: 129.9610 - val_loss: 126.4980
Epoch 70/100
 - 0s - loss: 129.9674 - val_loss: 126.3554
Epoch 71/100
 - 0s - loss: 129.9355 - val_loss: 126.5276
Epoch 72/100
 - 0s - loss: 129.9532 - val_loss: 126.5047
Epoch 73/100
 - 0s - loss: 130.0261 - val_loss: 126.3427
Epoch 74/100
 - 0s - loss: 129.8710 - val_loss: 126.4904
Epoch 75/100
 - 0s - loss: 129.9846 - val_loss: 126.5740
Epoch 76/100
 - 0s - loss: 129.8892 - val_loss: 126.5205
Epoch 77/100
 - 0s - loss: 129.9590 - val_loss: 126.5225
Epoch 78/100
 - 0s - loss: 129.9532 - val_loss: 126.6051
Epoch 79/100
 - 0s - loss: 129.9165 - val_loss: 126.6022
Epoch 80/100
 - 0s - loss: 129.8700 - val_loss: 126.4387
Epoch 81/100
 - 0s - loss: 129.9005 - val_loss: 126.4277
Epoch 82/100
 - 0s - loss: 129.9069 - val_loss: 126.4546
Epoch 83/100
 - 0s - loss: 129.8781 - val_lo

...

Epoch 80/100
 - 0s - loss: 122.8138 - val_loss: 138.3705
Epoch 81/100
 - 0s - loss: 122.8323 - val_loss: 138.3619
Epoch 82/100
 - 0s - loss: 122.7880 - val_loss: 138.4356
Epoch 83/100
 - 0s - loss: 122.7578 - val_loss: 138.3250
Epoch 84/100
 - 0s - loss: 122.8136 - val_loss: 138.4235
Epoch 85/100
 - 0s - loss: 122.7691 - val_loss: 138.7787
Epoch 86/100
 - 0s - loss: 122.7854 - val_loss: 138.6173
Epoch 87/100
 - 0s - loss: 122.7993 - val_loss: 138.7142
Epoch 88/100
 - 0s - loss: 122.7508 - val_loss: 138.7548
Epoch 89/100
 - 0s - loss: 122.7743 - val_loss: 138.6982
Epoch 90/100
 - 0s - loss: 122.7562 - val_loss: 138.9539
Epoch 91/100
 - 0s - loss: 122.8204 - val_loss: 138.7499
Epoch 92/100
 - 0s - loss: 122.6709 - val_loss: 138.9556
Epoch 93/100
 - 0s - loss: 122.7167 - val_loss: 139.2334
Epoch 94/100
 - 0s - loss: 122.7943 - val_loss: 139.1507
Epoch 95/100
 - 0s - loss: 122.7036 - val_loss: 139.1517
Epoch 96/100
 - 0s - loss: 122.6906 - val_loss: 138.9795
Epoch 97/100
 - 0s - loss: 122.

Epoch 20/100
 - 0s - loss: 125.1460 - val_loss: 132.7492
Epoch 21/100
 - 0s - loss: 125.2537 - val_loss: 132.6434
Epoch 22/100
 - 0s - loss: 125.1234 - val_loss: 132.8905
Epoch 23/100
 - 0s - loss: 125.2490 - val_loss: 132.4372
Epoch 24/100
 - 0s - loss: 125.0070 - val_loss: 132.9179
Epoch 25/100
 - 0s - loss: 125.0311 - val_loss: 133.0716
Epoch 26/100
 - 0s - loss: 125.0859 - val_loss: 132.9288
Epoch 27/100
 - 0s - loss: 125.1496 - val_loss: 133.6206
Epoch 28/100
 - 0s - loss: 125.0294 - val_loss: 133.3101
Epoch 29/100
 - 0s - loss: 124.9616 - val_loss: 133.1478
Epoch 30/100
 - 0s - loss: 124.9923 - val_loss: 133.2439
Epoch 31/100
 - 0s - loss: 124.9748 - val_loss: 133.4306
Epoch 32/100
 - 0s - loss: 125.0182 - val_loss: 133.2831
Epoch 33/100
 - 0s - loss: 125.0549 - val_loss: 133.5261
Epoch 34/100
 - 0s - loss: 124.8881 - val_loss: 133.4158
Epoch 35/100
 - 0s - loss: 124.9865 - val_loss: 133.0608
Epoch 36/100
 - 0s - loss: 124.9845 - val_loss: 133.3010
Epoch 37/100
 - 0s - loss: 124.

Epoch 62/100
 - 0s - loss: 125.5639 - val_loss: 130.7636
Epoch 63/100
 - 0s - loss: 125.6360 - val_loss: 131.1210
Epoch 64/100
 - 0s - loss: 125.5808 - val_loss: 130.7218
Epoch 65/100
 - 0s - loss: 125.5790 - val_loss: 130.7756
Epoch 66/100
 - 0s - loss: 125.5068 - val_loss: 131.0540
Epoch 67/100
 - 0s - loss: 125.5639 - val_loss: 131.0960
Epoch 68/100
 - 0s - loss: 125.6144 - val_loss: 131.0079
Epoch 69/100
 - 0s - loss: 125.5845 - val_loss: 130.8147
Epoch 70/100
 - 0s - loss: 125.5008 - val_loss: 130.9131
Epoch 71/100
 - 0s - loss: 125.5109 - val_loss: 130.7047
Epoch 72/100
 - 0s - loss: 125.5854 - val_loss: 130.8524
Epoch 73/100
 - 0s - loss: 125.5190 - val_loss: 130.8518
Epoch 74/100
 - 0s - loss: 125.4860 - val_loss: 131.2259
Epoch 75/100
 - 0s - loss: 125.5784 - val_loss: 131.1800
Epoch 76/100
 - 0s - loss: 125.5012 - val_loss: 130.9681
Epoch 77/100
 - 0s - loss: 125.4820 - val_loss: 131.0576
Epoch 78/100
 - 0s - loss: 125.5464 - val_loss: 130.6022
Epoch 79/100
 - 0s - loss: 125.

Epoch 2/100
 - 0s - loss: 125.0674 - val_loss: 132.8479
Epoch 3/100
 - 0s - loss: 124.8193 - val_loss: 133.4916
Epoch 4/100
 - 0s - loss: 124.5931 - val_loss: 133.7547
Epoch 5/100
 - 0s - loss: 124.6520 - val_loss: 134.5248
Epoch 6/100
 - 0s - loss: 124.3772 - val_loss: 134.7747
Epoch 7/100
 - 0s - loss: 124.1973 - val_loss: 134.6922
Epoch 8/100
 - 0s - loss: 124.0844 - val_loss: 134.9316
Epoch 9/100
 - 0s - loss: 123.9267 - val_loss: 135.1182
Epoch 10/100
 - 0s - loss: 123.8769 - val_loss: 135.2729
Epoch 11/100
 - 0s - loss: 123.8898 - val_loss: 135.8348
Epoch 12/100
 - 0s - loss: 123.6825 - val_loss: 135.6944
Epoch 13/100
 - 0s - loss: 123.7010 - val_loss: 135.7204
Epoch 14/100
 - 0s - loss: 123.6684 - val_loss: 135.9197
Epoch 15/100
 - 0s - loss: 123.5277 - val_loss: 136.2062
Epoch 16/100
 - 0s - loss: 123.6334 - val_loss: 136.0743
Epoch 17/100
 - 0s - loss: 123.4779 - val_loss: 136.2580
Epoch 18/100
 - 0s - loss: 123.4416 - val_loss: 136.3979
Epoch 19/100
 - 0s - loss: 123.6337 - v

Epoch 44/100
 - 0s - loss: 127.5482 - val_loss: 125.2049
Epoch 45/100
 - 0s - loss: 127.5147 - val_loss: 125.2383
Epoch 46/100
 - 0s - loss: 127.5335 - val_loss: 125.3121
Epoch 47/100
 - 0s - loss: 127.5491 - val_loss: 125.2165
Epoch 48/100
 - 0s - loss: 127.5390 - val_loss: 125.2704
Epoch 49/100
 - 0s - loss: 127.4849 - val_loss: 125.2588
Epoch 50/100
 - 0s - loss: 127.5131 - val_loss: 125.3417
Epoch 51/100
 - 0s - loss: 127.5027 - val_loss: 125.3415
Epoch 52/100
 - 0s - loss: 127.5250 - val_loss: 125.3501
Epoch 53/100
 - 0s - loss: 127.4441 - val_loss: 125.4293
Epoch 54/100
 - 0s - loss: 127.4817 - val_loss: 125.2231
Epoch 55/100
 - 0s - loss: 127.5005 - val_loss: 125.3682
Epoch 56/100
 - 0s - loss: 127.4800 - val_loss: 125.4159
Epoch 57/100
 - 0s - loss: 127.4337 - val_loss: 125.3846
Epoch 58/100
 - 0s - loss: 127.4868 - val_loss: 125.3747
Epoch 59/100
 - 0s - loss: 127.5384 - val_loss: 125.3367
Epoch 60/100
 - 0s - loss: 127.4214 - val_loss: 125.4615
Epoch 61/100
 - 0s - loss: 127.

In [50]:
# Calculate the length, the mean and the standard deviation
length_MSE_C = len(MSE_list_C)
mean_MSE_C = np.mean(MSE_list_C)
sd_MSE_C = np.std(MSE_list_C)

In [51]:
# Print the values
print(f'Number of mean squared errors: {length_MSE_C}')
print(f'Mean of the mean squared errors: {mean_MSE_C:.2f}')
print(f'Standard deviation of the mean squared errors: {sd_MSE_C:.2f}')

Number of mean squared errors: 50
Mean of the mean squared errors: 132.22
Standard deviation of the mean squared errors: 7.49


In [52]:
# initialise data of lists. 
data = {'A':[mean_MSE_A , sd_MSE_A], 'B':[mean_MSE_B , sd_MSE_B],
        'C':[mean_MSE_C , sd_MSE_C]} 
  
# Creates pandas DataFrame. 
df = pd.DataFrame(data, index =['Mean', 'StD'])

# print the data 
df.round(2)

,A,B,C
Mean,157.51,139.95,132.22
StD,16.01,16.01,7.49


__How does the mean of the mean squared errors compare to that from Step B?__

The mean of the mean squared errors for C is lower so better because of the a number of epochs higher.

<a id='item34'></a>

## D. Increase the number of hidden layers

Repeat part B but use a neural network with the following instead:
- Three hidden layers, each of 10 nodes and ReLU activation function.

In [53]:
# define regression model with 3 hidden layers, each of 10 nodes and ReLU activation function.
def regression_model_3hl():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [54]:
# build the model with regression_model_3hl
model_3hl = regression_model_3hl()

In [55]:
# Build the list of mean squared errors
MSE_list_D = calcul_MSE(model_3hl, predictors_norm, 50)

Train on 721 samples, validate on 309 samples
Epoch 1/50
 - 1s - loss: 1558.2794 - val_loss: 1599.9515
Epoch 2/50
 - 0s - loss: 1545.0840 - val_loss: 1588.8065
Epoch 3/50
 - 0s - loss: 1534.1135 - val_loss: 1578.1941
Epoch 4/50
 - 0s - loss: 1521.3303 - val_loss: 1563.3242
Epoch 5/50
 - 0s - loss: 1502.0203 - val_loss: 1540.5657
Epoch 6/50
 - 0s - loss: 1471.8224 - val_loss: 1504.5103
Epoch 7/50
 - 0s - loss: 1423.9765 - val_loss: 1448.6720
Epoch 8/50
 - 0s - loss: 1350.9286 - val_loss: 1365.4095
Epoch 9/50
 - 0s - loss: 1245.9255 - val_loss: 1246.6982
Epoch 10/50
 - 0s - loss: 1101.9878 - val_loss: 1090.6012
Epoch 11/50
 - 0s - loss: 927.7341 - val_loss: 901.1218
Epoch 12/50
 - 0s - loss: 736.3671 - val_loss: 701.6598
Epoch 13/50
 - 0s - loss: 560.4770 - val_loss: 529.1624
Epoch 14/50
 - 0s - loss: 435.5215 - val_loss: 405.1017
Epoch 15/50
 - 0s - loss: 356.1289 - val_loss: 331.0949
Epoch 16/50
 - 0s - loss: 302.5950 - val_loss: 288.2331
Epoch 17/50
 - 0s - loss: 268.0518 - val_loss: 

Epoch 43/50
 - 0s - loss: 153.1134 - val_loss: 142.6427
Epoch 44/50
 - 0s - loss: 154.0386 - val_loss: 142.4836
Epoch 45/50
 - 0s - loss: 153.0244 - val_loss: 142.8181
Epoch 46/50
 - 0s - loss: 153.0811 - val_loss: 142.4641
Epoch 47/50
 - 0s - loss: 153.1214 - val_loss: 143.2477
Epoch 48/50
 - 0s - loss: 152.9056 - val_loss: 142.4581
Epoch 49/50
 - 0s - loss: 152.7690 - val_loss: 142.8463
Epoch 50/50
 - 0s - loss: 153.2286 - val_loss: 142.6710
--------- MSE 3/50 = 142.67------------------------------------------
Train on 721 samples, validate on 309 samples
Epoch 1/50
 - 0s - loss: 149.7885 - val_loss: 149.2569
Epoch 2/50
 - 0s - loss: 149.9675 - val_loss: 148.9165
Epoch 3/50
 - 0s - loss: 149.6144 - val_loss: 149.2235
Epoch 4/50
 - 0s - loss: 149.5530 - val_loss: 150.0392
Epoch 5/50
 - 0s - loss: 149.4529 - val_loss: 150.1545
Epoch 6/50
 - 0s - loss: 149.2065 - val_loss: 150.3284
Epoch 7/50
 - 0s - loss: 149.1767 - val_loss: 150.9047
Epoch 8/50
 - 0s - loss: 149.1467 - val_loss: 150.2

Epoch 34/50
 - 0s - loss: 146.1541 - val_loss: 142.2245
Epoch 35/50
 - 0s - loss: 146.8550 - val_loss: 142.0132
Epoch 36/50
 - 0s - loss: 146.2687 - val_loss: 141.9761
Epoch 37/50
 - 0s - loss: 146.2237 - val_loss: 141.9588
Epoch 38/50
 - 0s - loss: 146.0370 - val_loss: 142.1602
Epoch 39/50
 - 0s - loss: 146.2677 - val_loss: 142.1527
Epoch 40/50
 - 0s - loss: 146.1936 - val_loss: 142.0385
Epoch 41/50
 - 0s - loss: 146.7613 - val_loss: 142.1632
Epoch 42/50
 - 0s - loss: 146.0516 - val_loss: 142.1210
Epoch 43/50
 - 0s - loss: 145.8600 - val_loss: 142.0516
Epoch 44/50
 - 0s - loss: 145.8562 - val_loss: 142.3418
Epoch 45/50
 - 0s - loss: 145.6978 - val_loss: 142.2075
Epoch 46/50
 - 0s - loss: 145.8726 - val_loss: 142.3284
Epoch 47/50
 - 0s - loss: 145.7653 - val_loss: 142.1636
Epoch 48/50
 - 0s - loss: 145.7467 - val_loss: 141.9201
Epoch 49/50
 - 0s - loss: 145.5425 - val_loss: 142.0358
Epoch 50/50
 - 0s - loss: 146.0007 - val_loss: 142.1781
--------- MSE 6/50 = 142.18---------------------

Epoch 25/50
 - 0s - loss: 128.8670 - val_loss: 155.9803
Epoch 26/50
 - 0s - loss: 128.7711 - val_loss: 156.1014
Epoch 27/50
 - 0s - loss: 128.6023 - val_loss: 155.7416
Epoch 28/50
 - 0s - loss: 128.4566 - val_loss: 156.0151
Epoch 29/50
 - 0s - loss: 128.4541 - val_loss: 155.5871
Epoch 30/50
 - 0s - loss: 128.7731 - val_loss: 155.1229
Epoch 31/50
 - 0s - loss: 128.1327 - val_loss: 155.9097
Epoch 32/50
 - 0s - loss: 128.2419 - val_loss: 155.8822
Epoch 33/50
 - 0s - loss: 128.1114 - val_loss: 156.0650
Epoch 34/50
 - 0s - loss: 128.0289 - val_loss: 156.0382
Epoch 35/50
 - 0s - loss: 128.1539 - val_loss: 155.6700
Epoch 36/50
 - 0s - loss: 128.0520 - val_loss: 155.7186
Epoch 37/50
 - 0s - loss: 128.0391 - val_loss: 156.5047
Epoch 38/50
 - 0s - loss: 128.0307 - val_loss: 156.0604
Epoch 39/50
 - 0s - loss: 128.0613 - val_loss: 156.2466
Epoch 40/50
 - 0s - loss: 127.8227 - val_loss: 156.1864
Epoch 41/50
 - 0s - loss: 127.7831 - val_loss: 155.6591
Epoch 42/50
 - 0s - loss: 128.1943 - val_loss: 1

Epoch 16/50
 - 0s - loss: 131.3923 - val_loss: 135.0146
Epoch 17/50
 - 0s - loss: 131.5254 - val_loss: 135.2790
Epoch 18/50
 - 0s - loss: 131.3226 - val_loss: 135.3126
Epoch 19/50
 - 0s - loss: 131.3310 - val_loss: 135.3513
Epoch 20/50
 - 0s - loss: 131.1700 - val_loss: 135.6167
Epoch 21/50
 - 0s - loss: 131.1432 - val_loss: 135.4841
Epoch 22/50
 - 0s - loss: 131.3676 - val_loss: 135.5989
Epoch 23/50
 - 0s - loss: 131.9098 - val_loss: 135.7397
Epoch 24/50
 - 0s - loss: 131.2773 - val_loss: 135.9967
Epoch 25/50
 - 0s - loss: 131.9827 - val_loss: 135.8889
Epoch 26/50
 - 0s - loss: 131.1646 - val_loss: 136.0345
Epoch 27/50
 - 0s - loss: 130.9698 - val_loss: 135.9260
Epoch 28/50
 - 0s - loss: 131.8021 - val_loss: 136.1020
Epoch 29/50
 - 0s - loss: 131.2890 - val_loss: 136.2770
Epoch 30/50
 - 0s - loss: 131.0590 - val_loss: 136.2335
Epoch 31/50
 - 0s - loss: 131.4485 - val_loss: 136.3740
Epoch 32/50
 - 0s - loss: 131.3795 - val_loss: 136.2033
Epoch 33/50
 - 0s - loss: 131.1110 - val_loss: 1

...

Epoch 39/50
 - 0s - loss: 118.2491 - val_loss: 126.6939
Epoch 40/50
 - 0s - loss: 117.9436 - val_loss: 127.4770
Epoch 41/50
 - 0s - loss: 117.8104 - val_loss: 127.2640
Epoch 42/50
 - 0s - loss: 117.8745 - val_loss: 127.0578
Epoch 43/50
 - 0s - loss: 117.8921 - val_loss: 127.3603
Epoch 44/50
 - 0s - loss: 117.6874 - val_loss: 127.0605
Epoch 45/50
 - 0s - loss: 117.9095 - val_loss: 126.9594
Epoch 46/50
 - 0s - loss: 117.8976 - val_loss: 127.4976
Epoch 47/50
 - 0s - loss: 117.8837 - val_loss: 128.0263
Epoch 48/50
 - 0s - loss: 117.6607 - val_loss: 127.2833
Epoch 49/50
 - 0s - loss: 117.6058 - val_loss: 127.1098
Epoch 50/50
 - 0s - loss: 118.1343 - val_loss: 127.4505
--------- MSE 37/50 = 127.45------------------------------------------
Train on 721 samples, validate on 309 samples
Epoch 1/50
 - 0s - loss: 123.9761 - val_loss: 113.4304
Epoch 2/50
 - 0s - loss: 123.2665 - val_loss: 113.3992
Epoch 3/50
 - 0s - loss: 122.9044 - val_loss: 114.9578
Epoch 4/50
 - 0s - loss: 122.6274 - val_loss: 

Epoch 30/50
 - 0s - loss: 122.4337 - val_loss: 115.5151
Epoch 31/50
 - 0s - loss: 122.6498 - val_loss: 116.3619
Epoch 32/50
 - 0s - loss: 122.5083 - val_loss: 115.5280
Epoch 33/50
 - 0s - loss: 122.7320 - val_loss: 116.6266
Epoch 34/50
 - 0s - loss: 122.2536 - val_loss: 115.2795
Epoch 35/50
 - 0s - loss: 122.1002 - val_loss: 116.2681
Epoch 36/50
 - 0s - loss: 122.2169 - val_loss: 116.4224
Epoch 37/50
 - 0s - loss: 122.2202 - val_loss: 116.3252
Epoch 38/50
 - 0s - loss: 122.0800 - val_loss: 115.9452
Epoch 39/50
 - 0s - loss: 122.3184 - val_loss: 116.4717
Epoch 40/50
 - 0s - loss: 122.2902 - val_loss: 116.7256
Epoch 41/50
 - 0s - loss: 122.1613 - val_loss: 116.9292
Epoch 42/50
 - 0s - loss: 122.5191 - val_loss: 116.1010
Epoch 43/50
 - 0s - loss: 123.8511 - val_loss: 116.4154
Epoch 44/50
 - 0s - loss: 122.9921 - val_loss: 116.6939
Epoch 45/50
 - 0s - loss: 122.2518 - val_loss: 116.0269
Epoch 46/50
 - 0s - loss: 121.8528 - val_loss: 117.4090
Epoch 47/50
 - 0s - loss: 122.1944 - val_loss: 1

Epoch 21/50
 - 0s - loss: 120.9430 - val_loss: 115.4861
Epoch 22/50
 - 0s - loss: 120.7483 - val_loss: 116.0355
Epoch 23/50
 - 0s - loss: 120.5424 - val_loss: 115.8424
Epoch 24/50
 - 0s - loss: 120.6787 - val_loss: 115.8099
Epoch 25/50
 - 0s - loss: 120.5517 - val_loss: 116.1137
Epoch 26/50
 - 0s - loss: 120.4662 - val_loss: 116.2142
Epoch 27/50
 - 0s - loss: 120.5453 - val_loss: 116.3532
Epoch 28/50
 - 0s - loss: 120.7109 - val_loss: 116.5487
Epoch 29/50
 - 0s - loss: 121.5216 - val_loss: 116.5266
Epoch 30/50
 - 0s - loss: 121.0233 - val_loss: 116.5583
Epoch 31/50
 - 0s - loss: 120.4349 - val_loss: 116.4332
Epoch 32/50
 - 0s - loss: 120.7103 - val_loss: 116.7010
Epoch 33/50
 - 0s - loss: 120.4845 - val_loss: 116.6377
Epoch 34/50
 - 0s - loss: 120.3301 - val_loss: 116.7791
Epoch 35/50
 - 0s - loss: 120.7790 - val_loss: 116.6437
Epoch 36/50
 - 0s - loss: 120.6998 - val_loss: 116.6227
Epoch 37/50
 - 0s - loss: 120.7185 - val_loss: 116.8116
Epoch 38/50
 - 0s - loss: 120.3200 - val_loss: 1

Epoch 12/50
 - 0s - loss: 120.5106 - val_loss: 114.5976
Epoch 13/50
 - 0s - loss: 120.8590 - val_loss: 114.7474
Epoch 14/50
 - 0s - loss: 121.0222 - val_loss: 115.3914
Epoch 15/50
 - 0s - loss: 120.3076 - val_loss: 115.1908
Epoch 16/50
 - 0s - loss: 120.2390 - val_loss: 115.8009
Epoch 17/50
 - 0s - loss: 120.7554 - val_loss: 115.4846
Epoch 18/50
 - 0s - loss: 120.6011 - val_loss: 116.0231
Epoch 19/50
 - 0s - loss: 120.3468 - val_loss: 115.8133
Epoch 20/50
 - 0s - loss: 119.9780 - val_loss: 115.7464
Epoch 21/50
 - 0s - loss: 120.6899 - val_loss: 116.3222
Epoch 22/50
 - 0s - loss: 120.3838 - val_loss: 116.2315
Epoch 23/50
 - 0s - loss: 119.8782 - val_loss: 116.3750
Epoch 24/50
 - 0s - loss: 120.2728 - val_loss: 116.5862
Epoch 25/50
 - 0s - loss: 120.1093 - val_loss: 116.2942
Epoch 26/50
 - 0s - loss: 119.6392 - val_loss: 116.4327
Epoch 27/50
 - 0s - loss: 119.9040 - val_loss: 116.5409
Epoch 28/50
 - 0s - loss: 119.9098 - val_loss: 116.7241
Epoch 29/50
 - 0s - loss: 120.3037 - val_loss: 1

Epoch 3/50
 - 0s - loss: 120.3541 - val_loss: 114.3220
Epoch 4/50
 - 0s - loss: 120.0905 - val_loss: 115.2550
Epoch 5/50
 - 0s - loss: 120.0115 - val_loss: 115.5763
Epoch 6/50
 - 0s - loss: 119.4713 - val_loss: 116.0736
Epoch 7/50
 - 0s - loss: 119.8019 - val_loss: 116.3965
Epoch 8/50
 - 0s - loss: 119.6904 - val_loss: 116.8117
Epoch 9/50
 - 0s - loss: 119.5200 - val_loss: 116.5030
Epoch 10/50
 - 0s - loss: 119.4192 - val_loss: 116.8070
Epoch 11/50
 - 0s - loss: 119.0912 - val_loss: 116.5824
Epoch 12/50
 - 0s - loss: 119.1562 - val_loss: 117.7245
Epoch 13/50
 - 0s - loss: 118.9271 - val_loss: 117.3018
Epoch 14/50
 - 0s - loss: 119.5424 - val_loss: 118.1469
Epoch 15/50
 - 0s - loss: 119.3697 - val_loss: 117.2693
Epoch 16/50
 - 0s - loss: 118.9935 - val_loss: 117.7412
Epoch 17/50
 - 0s - loss: 119.0512 - val_loss: 117.2179
Epoch 18/50
 - 0s - loss: 118.7703 - val_loss: 117.6225
Epoch 19/50
 - 0s - loss: 119.1706 - val_loss: 118.4584
Epoch 20/50
 - 0s - loss: 119.0894 - val_loss: 117.9863

In [56]:
# Calculate the length, the mean and the standard deviation
length_MSE_D = len(MSE_list_D)
mean_MSE_D = np.mean(MSE_list_D)
sd_MSE_D = np.std(MSE_list_D)

In [57]:
# Print the values
print(f'Number of mean squared errors: {length_MSE_D}')
print(f'Mean of the mean squared errors: {mean_MSE_D:.2f}')
print(f'Standard deviation of the mean squared errors: {sd_MSE_D:.2f}')

Number of mean squared errors: 50
Mean of the mean squared errors: 133.66
Standard deviation of the mean squared errors: 14.00


__How does the mean of the mean squared errors compare to that from Step B?__

The mean of the mean squared errors for D is lower so better because of the 3 hidden layers instead of 1 for B

<a id='item35'></a>

## E. Report 

In [58]:
# DataFrame

# initialise data of lists. 
data = {'A':[mean_MSE_A , sd_MSE_A], 'B':[mean_MSE_B , sd_MSE_B],
        'C':[mean_MSE_C , sd_MSE_C], 'D':[mean_MSE_D , sd_MSE_D]} 
  
# Creates pandas DataFrame. 
df = pd.DataFrame(data, index =['Mean', 'StD'])
df = df.round(2)
  
# print the data 
df 

,A,B,C,D
Mean,157.51,139.95,132.22,133.66
StD,16.01,16.01,7.49,14.00


***
__Best Model:        <font color=blue>C</font><br>__
- One hidden layer
- Data normalized
- 100 epochs
